In [1]:
import numpy as np
from tools import load, estimate_derivative, expand_and_sparsify, train, test

In [7]:
n_hd = 3000                                                         # HD layer size
n_out = 6                                                           # Number of categories
k = 10                                                              # k-WTA
p = 0.2                                                             # Output synapse flip probability -> chance of potentiation if pre&post active

file_train = 'mix_100_20_1'                                         # Training sequence
file_test = 'mix_50_20_1'                                           # Test sequence

### Load sensor data

In [8]:
train_data, train_sequence, train_times_sec, train_sequence_sec = load(file_train, reduced=True)
train_d_sensor_data = np.apply_along_axis(estimate_derivative, axis=0, arr=train_data)
train_data = np.hstack((train_data, train_d_sensor_data))           # Raw sensor train read-out sequence. (n_timesteps_train, n_dense = 16)

test_data, test_sequence, test_times_sec, test_sequence_sec = load(file_test, reduced=True)
test_d_sensor_data = np.apply_along_axis(estimate_derivative, axis=0, arr=test_data)
test_data = np.hstack((test_data, test_d_sensor_data))              # Raw sensor test read-out sequence. (n_timesteps, n_dense = 16)

### Expand-and-sparsify

In [9]:
W_hd = np.random.binomial(n=1, p=0.05, size=(n_hd, train_data.shape[1]))

z_hd_train = expand_and_sparsify(train_data, train_sequence, train_times_sec, train_sequence_sec, k=k, n_hd=n_hd, W_hd=W_hd)
z_hd_test = expand_and_sparsify(test_data, test_sequence, test_times_sec, test_sequence_sec, n_hd=n_hd, W_hd=W_hd)

### Run classification

In [10]:
W_hd, W_out = train(train_data, train_sequence, train_times_sec, train_sequence_sec,
                    n_hd=n_hd, n_out=n_out, k=k, p=p,
                    )

train_acc = test(train_data, train_sequence, train_times_sec, train_sequence_sec,
                 W_hd, W_out,
                 n_hd=n_hd, n_out=n_out, k=k, integration_delay=0.,
                 )

test_acc = test(test_data, test_sequence, test_times_sec, test_sequence_sec,
                W_hd, W_out,
                n_hd=n_hd, n_out=n_out, k=k, integration_delay=0.,
                )

print(f'Train accuracy: {train_acc:.4f}, Test accuracy: {test_acc:.4f}\n')

Train accuracy: 0.8283, Test accuracy: 0.6533

